# Trying out keras-tuner

In [1]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>

https://keras.io/keras_tuner/

https://github.com/keras-team/keras

# First example

https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html

In [1]:
import tensorflow as tf
import kerastuner as kt
import tensorflow_datasets as tfds

2022-06-03 15:22:16.006056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 15:22:16.006074: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/tmp/ipykernel_1919138/167059113.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


## Dataset

In [2]:
data = tfds.load("cifar10")
train_ds, test_ds = data["train"], data["test"]


def standardize_record(record):
    return tf.cast(record["image"], tf.float32) / 255.0, record["label"]


train_ds = train_ds.map(standardize_record).take(1024).batch(16).shuffle(100)
test_ds = test_ds.map(standardize_record).take(128).batch(16)

2022-06-03 15:22:24.026221: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-03 15:22:24.026242: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-03 15:22:24.026256: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (morricone.ipa.amolf.nl): /proc/driver/nvidia/version does not exist
2022-06-03 15:22:24.026456: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Model

In [3]:
def build_model(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    for i in range(hp.Int("conv_blocks", 3, 5, default=3)):
        filters = hp.Int("filters_" + str(i), 32, 256, step=32)
    for _ in range(2):
        x = tf.keras.layers.Convolution2D(filters, kernel_size=(3, 3), padding="same")(
            x
        )
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
    if hp.Choice("pooling_" + str(i), ["avg", "max"]) == "max":
        x = tf.keras.layers.MaxPool2D()(x)
    else:
        x = tf.keras.layers.AvgPool2D()(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
        hp.Int("hidden_size", 30, 100, step=10, default=50), activation="relu"
    )(x)
    x = tf.keras.layers.Dropout(hp.Float("dropout", 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

## Model tuner

https://keras.io/api/keras_tuner/tuners/random/

In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    seed=111,
    alpha=0.0001,
    beta=2.6,
    num_initial_points=5,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    overwrite=True,
)

In [7]:
tuner = CVBayesianTuner(
    hypermodel=build_model,
    n_split=4,
    objective="val_accuracy",
    max_trials=10,
    seed=111,
    alpha=0.0001,
    beta=2.6,
    num_initial_points=5,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    overwrite=True,
)

## Run

In [8]:
tuner.search(
    train_ds,
    validation_data=test_ds,
    epochs=30,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)],
)

Trial 10 Complete [00h 00m 25s]
val_accuracy: 0.328125

Best val_accuracy So Far: 0.328125
Total elapsed time: 00h 04m 31s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


## Results

In [6]:
best_model = tuner.get_best_models(1)[0]

In [7]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [8]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 64)       256       
 hNormalization)                                             

In [9]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
conv_blocks: 4
filters_0: 96
filters_1: 128
filters_2: 128
pooling_2: max
hidden_size: 80
dropout: 0.30000000000000004
learning_rate: 0.0001395510628578222
filters_3: 64
filters_4: 32
pooling_4: avg
pooling_3: avg
Score: 0.2265625
Trial summary
Hyperparameters:
conv_blocks: 4
filters_0: 32
filters_1: 32
filters_2: 160
pooling_2: max
hidden_size: 70
dropout: 0.30000000000000004
learning_rate: 0.0001
filters_3: 32
filters_4: 32
pooling_4: avg
pooling_3: avg
Score: 0.2109375
Trial summary
Hyperparameters:
conv_blocks: 5
filters_0: 128
filters_1: 32
filters_2: 96
pooling_2: max
hidden_size: 90
dropout: 0.1
learning_rate: 0.0026421929875193183
filters_3: 32
filters_4: 32
pooling_4: avg
Score: 0.1875
Trial summary
Hyperparameters:
conv_blocks: 3
filters_0: 128
filters_1: 32
filters_2: 128
pooling_2: max
hidden_size: 90
dropout: 0.4
learning_rate: 0.0001
filters_3: 32
filters_4: 32
pooling_4: a

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 12
conv_blocks (Int)
{'default': 3, 'conditions': [], 'min_value': 3, 'max_value': 5, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
pooling_2 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
hidden_size (Int)
{'default': 50, 'conditions': [], 'min_value': 30, 'max_value': 100, 'step': 10, 'sampling': None}
dropout (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}
filters_3 (Int)
{'

# Overwriting the hyperparameters

## Defining a new hyper-parameter space

### Ex 1:

In [12]:
hp = kt.HyperParameters()
hp.Int("conv_blocks", 6, 8, default=6)
hp.Int("hidden_size", 10, 20, step=2, default=10)
with hp.name_scope("scope1"):
    hp.Choice("choice", [4, 5, 6], default=5)
    with hp.name_scope("scope2"):
        hp.Choice("choice", [7, 8, 9], default=8)
    hp.Int("range", min_value=0, max_value=10, step=1, default=0)

hp.values

{'conv_blocks': 6,
 'hidden_size': 10,
 'scope1/choice': 5,
 'scope1/scope2/choice': 8,
 'scope1/range': 0}

### Ex 2:

In [13]:
hp = kt.HyperParameters()
hp.Int("conv_blocks", 6, 8, default=6)
hp.Int("hidden_size", 10, 20, step=2, default=10)

10

### Ex 3:

In [14]:
hp = kt.HyperParameters()
hp.Fixed("conv_blocks", 6)
hp.Fixed("hidden_size", 10)

10

## Running the tuner with a new set of hyperparameters

https://keras.io/api/keras_tuner/hyperparameters/#hyperparameters-class

In [31]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    seed=None,
    hyperparameters=hp,
    tune_new_entries=False,  # set to false to fix other values to their default
    allow_new_entries=True,  # always set to True
    overwrite=True,
)

In [32]:
tuner.search(
    train_ds,
    validation_data=test_ds,
    epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)],
)

Trial 1 Complete [00h 00m 07s]
val_accuracy: 0.1599999964237213

Best val_accuracy So Far: 0.1599999964237213
Total elapsed time: 00h 00m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


tune_new_entries: if False, values not specified are set to default

In [33]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
conv_blocks: 6
hidden_size: 10
filters_0: 32
filters_1: 32
filters_2: 32
filters_3: 32
filters_4: 32
filters_5: 32
pooling_5: avg
dropout: 0.5
learning_rate: 0.0001
Score: 0.1599999964237213


# Using the model loader to create a specific model

## Getting the default model

In [17]:
hp = kt.HyperParameters()
model = build_model(hp)

In [18]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                           

## Specifing some attributes

In [48]:
hp = kt.HyperParameters()
hp.Fixed("conv_blocks", 6)
model = build_model(hp)

# Cross validation with several models

In [43]:
def build_model_2(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    layer1 = tf.keras.layers.Dense(
        hp.Int("dense1", 5, 25, step=5, default=21), activation="relu"
    )(x)
    layer2 = tf.keras.layers.Dense(
        hp.Int("dense2", 5, 25, step=5, default=21), activation="relu"
    )(layer1)
    layer3 = tf.keras.layers.Flatten()(layer2)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(layer3)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [44]:
def build_model_1(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    layer1 = tf.keras.layers.Dense(
        hp.Int("dense", 32, 256, step=32, default=100), activation="relu"
    )(x)
    layer2 = tf.keras.layers.Flatten()(layer1)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(layer2)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float("learning_rate", 1e-4, 1e-2, sampling="log")
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [45]:
def build_model_total(hp):
    model_type = hp.Choice("model_type", ["type1", "type2"], default="type1")
    if model_type == "type1":
        with hp.conditional_scope("model_type", "type1"):
            model = build_model_1(hp)
    if model_type == "type2":
        with hp.conditional_scope("model_type", "type2"):
            model = build_model_2(hp)
    return model

In [46]:
tuner = kt.RandomSearch(
    build_model_total,
    objective="val_accuracy",
    max_trials=5,
    seed=None,
    tune_new_entries=True,
    allow_new_entries=True,  # always set to True
    overwrite=True,
)

In [47]:
tuner.search(
    train_ds,
    validation_data=test_ds,
    epochs=3,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)],
)

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.28999999165534973

Best val_accuracy So Far: 0.3400000035762787
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [48]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
model_type: type2
dense1: 5
dense2: 20
learning_rate: 0.00028884164416677734
Score: 0.3400000035762787
Trial summary
Hyperparameters:
model_type: type2
dense1: 21
dense2: 21
learning_rate: 0.0001
Score: 0.3100000023841858
Trial summary
Hyperparameters:
model_type: type1
dense: 128
learning_rate: 0.0018977027841734885
Score: 0.3100000023841858
Trial summary
Hyperparameters:
model_type: type1
dense: 64
learning_rate: 0.0011843198685147023
Score: 0.3100000023841858
Trial summary
Hyperparameters:
model_type: type1
dense: 32
learning_rate: 0.0013733775147488259
Score: 0.28999999165534973


# Adding tensorboard

TODO

# Bayesian optimisation

TODO

# Subclass to use cross validation

TODO

In [6]:
class CVBayesianTuner(kt.BayesianOptimization):
    """This tuner takes from the bayesian optimisation, but adds cross validation"""

    def __init__(self, *args, n_split=5, **kwargs):
        self.n_split = n_split
        super(CVBayesianTuner, self).__init__(*args, **kwargs)

In [ ]:
def run_trial(self, trial, *args, **kwargs):
    hp = trial.hyperparameters
    model = self.hypermodel.build(hp)
    return self.hypermodel.fit(hp, model, *args, **kwargs)

# Brouillon